# Install the necessary package

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


# Login wandb and hugging face

In [2]:
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login
user_secrets = UserSecretsClient()

login(token = user_secrets.get_secret("HUGGING_FACE_API_KEY"))
wandb.login(key=user_secrets.get_secret("wandb_api_key"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Import necessary libraries

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer

from torch.utils.data import DataLoader
import torch
import numpy as np
import collections
import evaluate
from evaluate import evaluator
from tqdm.auto import tqdm
import collections



2024-07-14 09:41:21.117638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 09:41:21.117772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 09:41:21.246055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load the dataset

In [4]:
source = "/kaggle/input/data-mash-covid"
data_files = {"train": f"{source}/dataset_train.csv", 
              "validation": f"{source}/dataset_val.csv", 
              "test_mash": f"{source}/dataset_test_mash.csv",
              "test_covid": f"{source}/dataset_test_covid.csv"}
ds = load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test_mash split: 0 examples [00:00, ? examples/s]

Generating test_covid split: 0 examples [00:00, ? examples/s]

In [5]:
def convert_text(batch):
  aux_list = []
  for x, y in zip(batch["answers"], batch["answers"]):
    my_dict = {"text":eval(x)["text"], "answer_start":eval(x)["answer_start"]}
    aux_list.append(my_dict)
    
  return {"texts":aux_list}



prepared_ds = ds.map(convert_text, batched=True)
prepared_ds = prepared_ds.remove_columns("answers")
prepared_ds = prepared_ds.rename_column("texts", "answers")
prepared_ds

Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10500
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1590
    })
    test_mash: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1307
    })
    test_covid: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 257
    })
})

In [6]:
max_length = 512
stride = 128
model_id = 'Eurosmart/bert-qa-mash-covid'
tokenizer = AutoTokenizer.from_pretrained(model_id)
def preprocess_validation(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

# Preprocess the test datasets

In [7]:

test_mash_dataset = prepared_ds["test_mash"].map(
    preprocess_validation,
    batched=True,
    remove_columns=prepared_ds["test_mash"].column_names,
)

test_covid_dataset = prepared_ds["test_covid"].map(
    preprocess_validation,
    batched=True,
    remove_columns=prepared_ds["test_covid"].column_names,
)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [8]:

n_best = 20
max_answer_length = 100


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [9]:
output_model = 'bert-qa-mash-covid'
data_collator = DefaultDataCollator()

model = AutoModelForQuestionAnswering.from_pretrained(model_id)

training_args = TrainingArguments(
    output_dir= output_model,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_ds['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Evaluate test covid

In [10]:
metric = evaluate.load("squad")

predictions, _, _ = trainer.predict(test_covid_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, test_covid_dataset, prepared_ds["test_covid"])

  0%|          | 0/257 [00:00<?, ?it/s]

{'exact_match': 17.120622568093385, 'f1': 48.67344996729952}

# Evaluate test mash

In [11]:
metric = evaluate.load("squad")

predictions, _, _ = trainer.predict(test_mash_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, test_mash_dataset, prepared_ds["test_mash"])

  0%|          | 0/1307 [00:00<?, ?it/s]

{'exact_match': 21.193573068094874, 'f1': 56.99707147476903}